# Создание чат-бота с использованием модели Seq2Seq

In [1]:
# Загрузка библиотек
from __future__ import print_function

import sys
import os
import pandas as pd
import numpy as np
import re
import nltk

from keras.layers import Input, Embedding, LSTM, TimeDistributed, Dense, Bidirectional
from keras.models import Model, load_model

INPUT_LENGTH = 20
OUTPUT_LENGTH = 20

# Загрузка и предварительная обработка данных:

* Следующим шагом является переформатирование нашего файла данных и загрузка данных в структуры, с которыми мы можем работать.
* Набор данных "Cornell Movie-Dialogs Corpus" состоит из следующего описания:
 * 220 579 разговоров между 10 292 парами героев фильмов
 * 9035 персонажей из 617 фильмов
 * 304,713 всего высказываний
 * Этот набор данных большой и разнообразный, и существует большое разнообразие языковых формальностей, периодов времени, настроений и т. д.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
conv_lines = open('./drive/My Drive/ChatBot_Dataset/movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')
lines = open('./drive/My Drive/ChatBot_Dataset/movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')

In [4]:
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [5]:
convs = []
for line in conv_lines[:-1]:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    convs.append(_line.split(','))

In [6]:
for k in convs[300]:
    print (k, id2line[k])

L3490 That's what he did to me.  He put cigarettes out on me.
L3491 Your father put cigarettes out on you?
L3492 Out on my back when I was a small boy.
L3493 Can I see your back?


In [7]:
# Sort the sentences into questions (inputs) and answers (targets)
questions = []
answers = []
for conv in convs:
    for i in range(len(conv)-1):
        questions.append(id2line[conv[i]])
        answers.append(id2line[conv[i+1]])
        
# Compare lengths of questions and answers
print(len(questions))
print(len(answers))

221616
221616


In [8]:
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|]", "", text)
#     text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = " ".join(text.split())
    return text

In [9]:
# Очищаем данные
clean_questions = []
for question in questions:
  clean_questions.append(clean_text(question))
clean_answers = []
for answer in answers:
  clean_answers.append(clean_text(answer))

In [10]:
# Найдем длину предложений (не используя nltk из-за скорости обработки)
lengths = []
# lengths.append([len(nltk.word_tokenize(sent)) for sent in clean_questions])
for question in clean_questions:
    lengths.append(len(question.split()))
for answer in clean_answers:
    lengths.append(len(answer.split()))
# Создадим dataframe, что бы можно было просмотреть значения данных.
lengths = pd.DataFrame(lengths, columns=['counts'])
print(np.percentile(lengths, 80))
print(np.percentile(lengths, 85))
print(np.percentile(lengths, 90))
print(np.percentile(lengths, 95))

16.0
19.0
24.0
32.0


In [11]:
# Удалим вопросы и ответы, которые короче 1 слова и длиннее 20 слов.
min_line_length = 2
max_line_length = 20

# Отфильтруем слишком длинные и короткие вопросы
short_questions_temp = []
short_answers_temp = []

for i, question in enumerate(clean_questions):
    if len(question.split()) >= min_line_length and len(question.split()) <= max_line_length:
        short_questions_temp.append(question)
        short_answers_temp.append(clean_answers[i])

# Отфильтруем слишком длинные и короткие вопросы
short_questions = []
short_answers = []

for i, answer in enumerate(short_answers_temp):
    if len(answer.split()) >= min_line_length and len(answer.split()) <= max_line_length:
        short_answers.append(answer)
        short_questions.append(short_questions_temp[i])
        
print(len(short_questions))
print(len(short_answers))

138528
138528


In [12]:
r = np.random.randint(1,len(short_questions))

for i in range(r, r+3):
    print(short_questions[i])
    print(short_answers[i])
    print()

burn them.
and the world the whole world.

and the world the whole world.
burn it all.

got a cause?
are the glory boys actually showing interest in investigationgs work? i may have a stroke.



In [13]:
import nltk
nltk.download('punkt')

num_samples = 20000 
short_questions = short_questions[:num_samples]
short_answers = short_answers[:num_samples]
#токенизация
short_questions_tok = [nltk.word_tokenize(sent) for sent in short_questions]
short_answers_tok = [nltk.word_tokenize(sent) for sent in short_answers]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [14]:
#train-validation split
data_size = len(short_questions_tok)

training_input  = short_questions_tok[:round(data_size*(80/100))]
training_input  = [tr_input[::-1] for tr_input in training_input] #обратная последовательность ввода для лучшей производительности
training_output = short_answers_tok[:round(data_size*(80/100))]

validation_input = short_questions_tok[round(data_size*(80/100)):]
validation_input  = [val_input[::-1] for val_input in validation_input] #обратная последовательность ввода для лучшей производительности
validation_output = short_answers_tok[round(data_size*(80/100)):]

print('training size', len(training_input))
print('validation size', len(validation_input))

training size 16000
validation size 4000


# encoder&decoder

In [15]:
vocab = {}
for question in short_questions_tok:
    for word in question:
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

for answer in short_answers_tok:
    for word in answer:
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1           

In [16]:
threshold = 15
count = 0
for k,v in vocab.items():
    if v >= threshold:
        count += 1

In [17]:
print("Size of total vocab:", len(vocab))
print("Size of vocab we will use:", count)

Size of total vocab: 13197
Size of vocab we will use: 1447


In [18]:
WORD_CODE_START = 1
WORD_CODE_PADDING = 0


word_num  = 2 
encoding = {}
decoding = {1: 'START'}
for word, count in vocab.items():
    if count >= threshold:
        encoding[word] = word_num 
        decoding[word_num ] = word
        word_num += 1

print("No. of vocab used:", word_num)

No. of vocab used: 1449


In [19]:
decoding[len(encoding)+2] = '<UNK>'
encoding['<UNK>'] = len(encoding)+2

In [20]:
dict_size = word_num+1
dict_size

1450

In [21]:
# Vectorizing dataset
def transform(encoding, data, vector_size=20):
    transformed_data = np.zeros(shape=(len(data), vector_size))
    for i in range(len(data)):
        for j in range(min(len(data[i]), vector_size)):
            try:
                transformed_data[i][j] = encoding[data[i][j]]
            except:
                transformed_data[i][j] = encoding['<UNK>']
    return transformed_data

In [22]:
#encoding training set
encoded_training_input = transform(
    encoding, training_input, vector_size=INPUT_LENGTH)
encoded_training_output = transform(
    encoding, training_output, vector_size=OUTPUT_LENGTH)

print('encoded_training_input', encoded_training_input.shape)
print('encoded_training_output', encoded_training_output.shape)

encoded_training_input (16000, 20)
encoded_training_output (16000, 20)


In [23]:
#encoding validation set
encoded_validation_input = transform(
    encoding, validation_input, vector_size=INPUT_LENGTH)
encoded_validation_output = transform(
    encoding, validation_output, vector_size=OUTPUT_LENGTH)

print('encoded_validation_input', encoded_validation_input.shape)
print('encoded_validation_output', encoded_validation_output.shape)

encoded_validation_input (4000, 20)
encoded_validation_output (4000, 20)


# Построение модели


# Sequence-to-Sequence

In [24]:
import tensorflow as tf
tf.keras.backend.clear_session()

In [25]:
INPUT_LENGTH = 20
OUTPUT_LENGTH = 20

encoder_input = Input(shape=(INPUT_LENGTH,))
decoder_input = Input(shape=(OUTPUT_LENGTH,))

In [26]:
from keras.layers import SimpleRNN

encoder = Embedding(dict_size, 128, input_length=INPUT_LENGTH, mask_zero=True)(encoder_input)
encoder = LSTM(512, return_sequences=True, unroll=True)(encoder)
encoder_last = encoder[:,-1,:]

print('encoder', encoder)
print('encoder_last', encoder_last)

decoder = Embedding(dict_size, 128, input_length=OUTPUT_LENGTH, mask_zero=True)(decoder_input)
decoder = LSTM(512, return_sequences=True, unroll=True)(decoder, initial_state=[encoder_last, encoder_last])

print('decoder', decoder)

encoder KerasTensor(type_spec=TensorSpec(shape=(None, 20, 512), dtype=tf.float32, name=None), name='lstm/transpose_2:0', description="created by layer 'lstm'")
encoder_last KerasTensor(type_spec=TensorSpec(shape=(None, 512), dtype=tf.float32, name=None), name='tf.__operators__.getitem/strided_slice:0', description="created by layer 'tf.__operators__.getitem'")
decoder KerasTensor(type_spec=TensorSpec(shape=(None, 20, 512), dtype=tf.float32, name=None), name='lstm_1/transpose_2:0', description="created by layer 'lstm_1'")


# Attention 

In [27]:
from keras.layers import Activation, dot, concatenate

attention = dot([decoder, encoder], axes=[2, 2])
attention = Activation('softmax', name='attention')(attention)
print('attention', attention)

context = dot([attention, encoder], axes=[2,1])
print('context', context)

decoder_combined_context = concatenate([context, decoder])
print('decoder_combined_context', decoder_combined_context)

output = TimeDistributed(Dense(512, activation="tanh"))(decoder_combined_context)
output = TimeDistributed(Dense(dict_size, activation="softmax"))(output)
print('output', output)

attention KerasTensor(type_spec=TensorSpec(shape=(None, 20, 20), dtype=tf.float32, name=None), name='attention/Softmax:0', description="created by layer 'attention'")
context KerasTensor(type_spec=TensorSpec(shape=(None, 20, 512), dtype=tf.float32, name=None), name='dot_1/MatMul:0', description="created by layer 'dot_1'")
decoder_combined_context KerasTensor(type_spec=TensorSpec(shape=(None, 20, 1024), dtype=tf.float32, name=None), name='concatenate/concat:0', description="created by layer 'concatenate'")
output KerasTensor(type_spec=TensorSpec(shape=(None, 20, 1450), dtype=tf.float32, name=None), name='time_distributed_1/Reshape_1:0', description="created by layer 'time_distributed_1'")


In [28]:
model = Model(inputs=[encoder_input, decoder_input], outputs=[output])
model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 20, 128)      185600      input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 20)]         0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 20, 512)      1312768     embedding[0][0]                  
______________________________________________________________________________________________

In [29]:
training_encoder_input = encoded_training_input
training_decoder_input = np.zeros_like(encoded_training_output)
training_decoder_input[:, 1:] = encoded_training_output[:,:-1]
training_decoder_input[:, 0] = WORD_CODE_START
training_decoder_output = np.eye(dict_size)[encoded_training_output.astype('int')]

validation_encoder_input = encoded_validation_input
validation_decoder_input = np.zeros_like(encoded_validation_output)
validation_decoder_input[:, 1:] = encoded_validation_output[:,:-1]
validation_decoder_input[:, 0] = WORD_CODE_START
validation_decoder_output = np.eye(dict_size)[encoded_validation_output.astype('int')]

In [30]:
model.fit(x=[training_encoder_input, training_decoder_input], y=[training_decoder_output],
          validation_data=([validation_encoder_input, validation_decoder_input], [validation_decoder_output]),
          #validation_split=0.05,
          batch_size=64, epochs=100)

Epoch 1/100
250/250 [==============================] - 32s 89ms/step - loss: 0.0023 - val_loss: 0.0020
Epoch 2/100
250/250 [==============================] - 21s 82ms/step - loss: 0.0020 - val_loss: 0.0019
Epoch 3/100
250/250 [==============================] - 21s 82ms/step - loss: 0.0019 - val_loss: 0.0018
Epoch 4/100
250/250 [==============================] - 20s 82ms/step - loss: 0.0018 - val_loss: 0.0017
Epoch 5/100
250/250 [==============================] - 20s 82ms/step - loss: 0.0018 - val_loss: 0.0017
Epoch 6/100
250/250 [==============================] - 20s 82ms/step - loss: 0.0017 - val_loss: 0.0017
Epoch 7/100
250/250 [==============================] - 20s 82ms/step - loss: 0.0017 - val_loss: 0.0017
Epoch 8/100
250/250 [==============================] - 21s 82ms/step - loss: 0.0017 - val_loss: 0.0017
Epoch 9/100
250/250 [==============================] - 21s 82ms/step - loss: 0.0017 - val_loss: 0.0016
Epoch 10/100
250/250 [==============================] - 21s 82ms/step - l

In [31]:
def prediction(raw_input):
    clean_input = clean_text(raw_input)
    input_tok = [nltk.word_tokenize(clean_input)]
    input_tok = [input_tok[0][::-1]]  #reverseing input seq
    encoder_input = transform(encoding, input_tok, 20)
    decoder_input = np.zeros(shape=(len(encoder_input), OUTPUT_LENGTH))
    decoder_input[:,0] = WORD_CODE_START
    for i in range(1, OUTPUT_LENGTH):
        output = model.predict([encoder_input, decoder_input]).argmax(axis=2)
        decoder_input[:,i] = output[:,i]
    return output

def decode(decoding, vector):
    text = ''
    for i in vector:
        if i == 0:
            break
        text += ' '
        text += decoding[i]
    return text

In [32]:
for i in range(20):
    seq_index = np.random.randint(1, len(short_questions))
    output = prediction(short_questions[seq_index])
    print ('Questions:', short_questions[seq_index])
    print ('Answers:', decode(decoding, output[0]))

Questions: because i like to torture you.
Answers:  oh , bianca ? can you get me my <UNK> like my lives ?
Questions: that is correct, sir.
Answers:  sorry , doolittle . i have <UNK> so much since i have been in here . so much . .
Questions: what are you doing, keitel blacksmith?
Answers:  get away , <UNK> .
Questions: you have tranquilizers?
Answers:  i told you ! i have got everything !
Questions: go ahead, bird dog.
Answers:  you are all clear .
Questions: he is not?
Answers:  alex , has david <UNK> in his <UNK> <UNK> ?
Questions: i am the stenographer.
Answers:  in other words , no .
Questions: what money?
Answers:  do not bullshit me .
Questions: now. one thing we need to discuss is timing. timing is absolutely crucial. what are you doing? anthony!
Answers:  nothing . go ahead .
Questions: guess it was not the pizza delivery guy.
Answers:  you are lucky . i almost pulled the <UNK> .
Questions: if it would not be too much of a bother... a little bite to eat would be...
Answers:  oh 